In [1]:
from pathlib import Path

import util
from util import p_, workflow

import automech

file = util.notebook_file() if util.is_notebook() else __file__
tag = util.file_tag(file)
root_path = Path("..")
par_mech = workflow.read_parent_mechanism(root_path=root_path)

In [2]:
# Generate submechanism
from automol.graph import enum

gen_mech1 = workflow.previous_version_species(tag=tag, root_path=root_path)
#  - Migrations
gen_mech1 = automech.enumerate_reactions(
    gen_mech1, enum.ReactionSmarts.qooh_formation, src_mech=par_mech
)
#   - Ring-forming scissions
gen_mech1 = automech.enumerate_reactions(
    gen_mech1,
    enum.ReactionSmarts.qooh_ring_forming_scission,
    src_mech=par_mech,
    excl_rcts=automech.unstable_species_names(gen_mech1),
)
#  - Ring-opening scissions
gen_mech1 = automech.enumerate_reactions(
    gen_mech1,
    enum.ReactionSmarts.ring_beta_scission,
    src_mech=par_mech,
    rcts_=[automech.species.names(gen_mech1.species, fml="C*H*O2")],
    excl_rcts=automech.unstable_species_names(gen_mech1),
)
#  - Beta scissions
gen_mech1 = automech.enumerate_reactions(
    gen_mech1,
    enum.ReactionSmarts.beta_scission,
    src_mech=par_mech,
    rcts_=[automech.species.names(gen_mech1.species, fml="C*H*O2")],
    excl_rcts=automech.unstable_species_names(gen_mech1),
)
#  - Vinyl beta scissions
gen_mech1 = automech.enumerate_reactions(
    gen_mech1,
    enum.ReactionSmarts.vinyl_beta_scission,
    src_mech=par_mech,
    rcts_=[automech.species.names(gen_mech1.species, fml="C*H*O2")],
    excl_rcts=automech.unstable_species_names(gen_mech1),
)
#  - Instabilities
gen_mech1 = automech.enumerate_products(
    gen_mech1,
    enum.ReactionSmarts.qooh_instability,
    src_mech=par_mech,
    rcts_=[automech.species.names(gen_mech1.species, fml="C*H*O2")],
)
automech.display(gen_mech1, height="1000px")

In [3]:
# Expand stereochemistry
ste_mech1, *_, gen_mech1 = workflow.expand_stereo(
    mech=gen_mech1, tag=tag, root_path=root_path
)

#  - Drop stereo-specific reactions
ste_mech1 = automech.drop_reactions_by_smiles(
    ste_mech1,
    rxn_smis=[
        # >> S(731)r0 = S(2258)z + C2H4(52) (Y67)
        r"C=C[C@@H](C[CH2])OO>>[CH2]/C=C\OO.C=C",
        # >> S(731)r1 = S(2258)z + C2H4(52) (Y67)
        r"C=C[C@H](C[CH2])OO>>[CH2]/C=C\OO.C=C",
    ],
)
automech.display(ste_mech1)


Expanding stereochemistry...


  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]


Stereoexpansion errors:


  0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
# Update previous version
gen_mech, ste_mech = workflow.update_previous_version(
    gen_mech=gen_mech1, ste_mech=ste_mech1, tag=tag, root_path=root_path
)
automech.display(gen_mech, height="1000px")

In [5]:
# Prepare calculation
workflow.prepare_calculation(
    gen_mech=gen_mech, ste_mech=ste_mech, tag=tag, root_path=root_path
)


Sorting mechanism...

Writing mechanism...
../data/D_r-o2_0a_v1_gen.json
../data/D_r-o2_0a_v1_ste.json
../data/mechanalyzer/D_r-o2_0a_v1_ste.dat
../data/mechanalyzer/D_r-o2_0a_v1_ste.csv


  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

In [6]:
# # Gather statistics
# workflow.gather_statistics(tag=tag, root_path=root_path)

In [7]:
# # Prepare simulation
# workflow.prepare_simulation(tag=tag, root_path=root_path)
# workflow.prepare_simulation_species(tag=tag, root_path=root_path)

In [8]:
# # Check rates
# workflow.plot_rates(tag=tag, root_path=root_path)

In [9]:
# # Plot simulation
# chart_dct = workflow.plot_simulation(
#     tag=tag,
#     x_col="O2_molecules",
#     root_path=root_path,
#     line_source_=["Hill", "Lokachari"],
#     point_source="Experiment",
#     control=False,
# )
# mech = automech.io.read(
#     p_.full_calculated_mechanism(tag, "json", path=p_.data(root_path))
# )
# for name, chart in chart_dct.items():
#     automech.display_species(mech, [name])
#     chart.show()